In [175]:
import pandas as pd

In [176]:
transactions = pd.read_excel('template.xlsx')
transactions = transactions.astype({'stock_code': str, 'description': str})
transactions['description'] = transactions['description'].str.upper()
transactions['stock_code'] = transactions['stock_code'].str.upper()
transactions['description'] = transactions['description'].str.replace('\bNAN\b', 'UNKNOWN', regex=True)
transactions.head()

,invoice_no,stock_code,description,quantity,price
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2.55
1,536365,71053,WHITE METAL LANTERN,6,3.39
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2.75
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,3.39
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,3.39


In [177]:
description = transactions.loc[:, ['stock_code', 'description']]
description.head()

,stock_code,description
0,85123A,WHITE HANGING HEART T-LIGHT HOLDER
1,71053,WHITE METAL LANTERN
2,84406B,CREAM CUPID HEARTS COAT HANGER
3,84029G,KNITTED UNION FLAG HOT WATER BOTTLE
4,84029E,RED WOOLLY HOTTIE WHITE HEART.


In [178]:
description = description.drop_duplicates(subset='stock_code')
description = description[~description['stock_code'].str.contains(r'POST|\bD\b|\bS\b|\bM\b|C2|AMAZONFEE|BANK CHARGES|DOT|^GIFT_', regex=True)]
description.shape

(2895, 2)

In [179]:
month_quantity = transactions.loc[:, ['stock_code', 'quantity']].groupby('stock_code').sum().reset_index()
month_quantity.head()

,stock_code,quantity
0,10002,422
1,10120,16
2,10123C,1
3,10124A,4
4,10124G,5


In [180]:
max_quantity = transactions.loc[:, ['stock_code', 'quantity']].groupby('stock_code').max().reset_index()
max_quantity.shape

(2907, 2)

In [181]:
avg_price = transactions.loc[:, ['stock_code', 'price']].groupby('stock_code').mean().reset_index()
avg_price = avg_price.rename(columns={'price': 'avg_price'})
avg_price.shape

(2907, 2)

In [182]:
selling = transactions.loc[:, ['invoice_no', 'stock_code', 'quantity']]
selling = selling.groupby(['invoice_no', 'stock_code']).count()
selling = selling['quantity'].groupby('stock_code').count().to_frame()
selling = selling.rename(columns={'quantity': 'selling'}).reset_index()
selling.shape

(2907, 2)

In [183]:
quantity = month_quantity.copy()
quantity.loc[(quantity['quantity'] < 0), 'quantity'] = quantity['quantity']
quantity.loc[(quantity['quantity'] < 30) & (max_quantity['quantity'] > 10), 'quantity'] = 1
quantity.shape
quantity.to_excel('quantity.xlsx', index=False)

In [184]:
clean = description.merge(quantity, on='stock_code', how='left')
clean = clean.merge(selling, on='stock_code', how='left')
clean = clean.merge(avg_price, on='stock_code', how='left').sort_values(by='stock_code')
clean.shape

(2895, 5)

In [185]:
clean.head()

,stock_code,description,quantity,selling,avg_price
31,10002,INFLATABLE POLITICAL GLOBE,422,44,1.125455
1624,10120,DOGGY RUBBER,1,4,0.210000
1623,10123C,HEARTS WRAPPING TAPE,1,1,0.650000
2136,10124A,SPOTS ON RED BOOKCOVER TAPE,4,1,0.420000
1467,10124G,ARMY CAMO BOOKCOVER TAPE,5,1,0.420000


In [186]:
clean.to_excel('clean.xlsx', index=False)